In [1]:
from onnxruntime.quantization.quantize import quantize_dynamic
from segment_anything.utils.onnx import SamOnnxModel
from onnxruntime.quantization import QuantType
from segment_anything import SamPredictor
from segment_anything import build_sam
import onnxruntime
import numpy as np
import torch
import cv2

In [2]:
model        = build_sam('checkpoint/sam_vit_h.pth')
onnx_model   = SamOnnxModel(model = model, return_single_mask = True)
dynamic_axes = {
                    'point_coords' : {1 : 'num_points'},
                    'point_labels' : {1 : 'num_points'}
              
                }

embed_dim  = model.prompt_encoder.embed_dim
embed_size = model.prompt_encoder.image_embedding_size

mask_input_size = [4 * x for x in embed_size]
dummy_inputs    = {
                        'image_embeddings' : torch.randn(1, embed_dim, *embed_size, dtype = torch.float),
                        'point_coords'     : torch.randint(low = 0, high = 1024, size = (1, 5, 2), dtype = torch.float),
                        'point_labels'     : torch.randint(low = 0, high = 4, size = (1, 5), dtype = torch.float),
                        'mask_input'       : torch.randn(1, 1, *mask_input_size, dtype = torch.float),
                        'has_mask_input'   : torch.tensor([1], dtype=torch.float),
                        'orig_im_size'     : torch.tensor([1500, 2250])
                    }

_ = onnx_model(**dummy_inputs)
output_names = ['masks', 'iou_predictions', 'low_res_masks']

In [3]:
onnx_model_path  = 'checkpoint/onnx_sam.onnx'
quant_model_path = 'checkpoint/quant_sam.onnx' 

with open('checkpoint/onnx_sam.onnx', 'wb') as f:
    
    torch.onnx.export(
                        onnx_model, tuple(dummy_inputs.values()),
                        f, export_params = True, verbose = False, opset_version = 16,
                        do_constant_folding = True, input_names = list(dummy_inputs.keys()),
                        output_names = output_names, dynamic_axes = dynamic_axes
                    )

/opt/conda/envs/torch/lib/python3.10/site-packages/segment_anything/modeling/transformer.py:232: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  attn = attn / math.sqrt(c_per_head)
/opt/conda/envs/torch/lib/python3.10/site-packages/segment_anything/utils/onnx.py:97: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  score_reweight = torch.tensor(
/opt/conda/envs/torch/lib/python3.10/site-packages/torch/onnx/symbolic_opset9.py:5589: UserWarning: Exporting aten::index operator of advanced indexing in opset 16 is achieved by c

============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [4]:
quantize_dynamic(
    model_input    = onnx_model_path,
    model_output   = quant_model_path,
    optimize_model = True,
    per_channel    = False, 
    reduce_range   = False,
    weight_type    = QuantType.QUInt8
)

onnx_model_path = quant_model_path
onnx_model_path

Ignore MatMul due to non constant B: /[/transformer/layers.0/self_attn/MatMul]
Ignore MatMul due to non constant B: /[/transformer/layers.0/self_attn/MatMul_1]
Ignore MatMul due to non constant B: /[/transformer/layers.0/cross_attn_token_to_image/MatMul]
Ignore MatMul due to non constant B: /[/transformer/layers.0/cross_attn_token_to_image/MatMul_1]
Ignore MatMul due to non constant B: /[/transformer/layers.0/cross_attn_image_to_token/MatMul]
Ignore MatMul due to non constant B: /[/transformer/layers.0/cross_attn_image_to_token/MatMul_1]
Ignore MatMul due to non constant B: /[/transformer/layers.1/self_attn/MatMul]
Ignore MatMul due to non constant B: /[/transformer/layers.1/self_attn/MatMul_1]
Ignore MatMul due to non constant B: /[/transformer/layers.1/cross_attn_token_to_image/MatMul]
Ignore MatMul due to non constant B: /[/transformer/layers.1/cross_attn_token_to_image/MatMul_1]
Ignore MatMul due to non constant B: /[/transformer/layers.1/cross_attn_image_to_token/MatMul]
Ignore Ma

'checkpoint/quant_sam.onnx'

In [5]:
ort_session = onnxruntime.InferenceSession(onnx_model_path)

model.to(device = 'cuda')
predictor = SamPredictor(model)

/opt/conda/envs/torch/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [6]:
image = cv2.cvtColor(cv2.imread('checkpoint/src_image.jpg'), cv2.COLOR_BGR2RGB)
predictor.set_image(image)

In [7]:
image_embedding = predictor.get_image_embedding().cpu().numpy()
image_embedding.shape

(1, 256, 64, 64)

In [8]:
input_point = np.array([[500, 375]])
input_label = np.array([1])

onnx_coord  = np.concatenate([input_point, np.array([[0.0, 0.0]])], axis = 0)[None, :, :]
onnx_label  = np.concatenate([input_label, np.array([-1])], axis = 0)[None, :].astype(np.float32)
onnx_coord  = predictor.transform.apply_coords(onnx_coord, image.shape[:2]).astype(np.float32)

onnx_mask_input     = np.zeros((1, 1, 256, 256), dtype = np.float32)
onnx_has_mask_input = np.zeros(1, dtype = np.float32) 